In [1]:
# https://www.kaggle.com/datasets/jawadali1045/20k-multi-class-crop-disease-images
!pip install opencv-python-headless
!pip install scikit-image

import numpy as np
import pandas as pd
import cv2
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from skimage import feature
import matplotlib.pyplot as plt


In [ ]:
healthy_path = '/content/dataset/healthy/*.png'
diseased_path = '/content/dataset/diseased/*.png'

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in glob.glob(folder):
        img = cv2.imread(filename)
        img = cv2.resize(img, (64, 64))  # Resize to a smaller size
        if img is not None:
            images.append(img)
            labels.append(label)
    return images, labels

healthy_images, healthy_labels = load_images_from_folder(healthy_path, 'healthy')
diseased_images, diseased_labels = load_images_from_folder(diseased_path, 'diseased')

images = np.array(healthy_images + diseased_images)
labels = np.array(healthy_labels + diseased_labels)

unique_classes, counts = np.unique(labels, return_counts=True)
class_distribution = dict(zip(unique_classes, counts))
print("Class distribution:", class_distribution)


In [ ]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = feature.hog(gray_image, pixels_per_cell=(8, 8),
                               cells_per_block=(2, 2), visualize=False)
        hog_features.append(features)
    return np.array(hog_features)

X = extract_hog_features(images)
y = labels


In [ ]:
unique_classes, counts = np.unique(labels, return_counts=True)
class_distribution = dict(zip(unique_classes, counts))
print("Class distribution:", class_distribution)


In [ ]:
healthy_images, healthy_labels = load_images_from_folder(healthy_path, 'healthy')
diseased_images, diseased_labels = load_images_from_folder(diseased_path, 'diseased')

print(f"Number of healthy images: {len(healthy_images)}")
print(f"Number of diseased images: {len(diseased_images)}")

images = np.array(healthy_images + diseased_images)
labels = np.array(healthy_labels + diseased_labels)

print(f"Total images loaded: {len(images)}")


In [ ]:
import matplotlib.pyplot as plt

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in glob.glob(folder):
        img = cv2.imread(filename)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            images.append(img)
            labels.append(label)
    return images, labels

healthy_images, healthy_labels = load_images_from_folder(healthy_path, 'healthy')
diseased_images, diseased_labels = load_images_from_folder(diseased_path, 'diseased')

print(f"Number of healthy images: {len(healthy_images)}")
print(f"Number of diseased images: {len(diseased_images)}")

images = np.array(healthy_images + diseased_images)
labels = np.array(healthy_labels + diseased_labels)

print(f"Total images loaded: {len(images)}")

if len(images) > 0:
    plt.figure(figsize=(10, 5))
    for i in range(min(5, len(images))):
        plt.subplot(1, 5, i + 1)
        plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        plt.axis('off')
    plt.show()
else:
    print("No images were loaded. Please check the dataset paths.")


In [ ]:
# Encode labels: healthy as 1, diseased as 0
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

svm_model = make_pipeline(SVC(kernel='linear'))
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

def predict_image(image):
    image_resized = cv2.resize(image, (64, 64))
    features = extract_hog_features([image_resized])
    prediction = svm_model.predict(features)
    return prediction[0]

new_image_path = '/content/dataset/healthy/h1.png'
new_image = cv2.imread(new_image_path)

if new_image is not None:
    result = predict_image(new_image)
    print("Predicted result for the new image:", result)
else:
    print("Could not load the image. Please check the path.")

In [ ]:
new_image_path = '/content/dataset/diseased/d2.png'
new_image = cv2.imread(new_image_path)

if new_image is not None:
    result = predict_image(new_image)
    print("Predicted result for the new image:", result)
else:
    print("Could not load the image. Please check the path.")